In [ ]:
import datetime as dt
import tempfile
import zipfile

from itertools import product, groupby
from pathlib import Path

import datazimmer as dz
import geopandas as gpd
import numpy as np
import pandas as pd

from netCDF4 import Dataset
from tqdm.notebook import tqdm

In [ ]:
dump_tmp_dir = Path.home() / "tmp" / "weather-dump-cache"
dump_tmp_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
nuts3_gdf = gpd.read_file(
    dump_tmp_dir / "NUTS_RG_01M_2021_4326.shp.zip", crs="EPSG:4326"
).loc[lambda df: df["LEVL_CODE"] == 3]

In [ ]:
tcol, lonc, latc = mcols = ["time", "longitude", "latitude"]
ind_names = [latc, lonc]

In [ ]:
# copernicus export
zip_path = (
    dump_tmp_dir
    / "dataset-insitu-gridded-observations-europe-31d0017a-a2ee-41a8-afda-90492da8d8b2.zip"
)

In [ ]:
dump_dir = dump_tmp_dir / "nuts-level-dumps"
dump_dir.mkdir(exist_ok=True)

In [ ]:
def dump_out(dfs, i, out_desc, dump_dir):
    gdf = (
        pd.concat(dfs)
        .pipe(
            lambda df: gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df[lonc], df[latc]),
                crs="EPSG:4326",  # "EPSG:4326"  / 3035
            )
        )
        .drop([lonc, latc], axis=1)
    )

    nuts_mean_df = (
        gpd.sjoin(nuts3_gdf, gdf, how="inner", predicate="contains")
        .groupby(["NUTS_ID", "day"])[out_desc]
        .mean()
        .reset_index()
    )
    nuts_mean_df.to_parquet(dump_dir / f"{out_desc}-{i}.parquet")

In [ ]:
with zipfile.ZipFile(zip_path) as zfp, tempfile.TemporaryDirectory() as tmpdir:
    for f in tqdm(zfp.filelist):
        zfp.extract(f, tmpdir)
        ofile = Path(tmpdir, f.filename)
        rootgrp = Dataset(ofile, "r", format="NETCDF4")

        out_col = next(filter(lambda e: e not in mcols, rootgrp.variables.keys()))

        out_var = rootgrp.variables[out_col]
        out_desc = out_var.long_name.replace(" ", "_").lower()

        geo_ind = pd.MultiIndex.from_tuples(
            product(*[rootgrp.variables[c][:].data for c in ind_names]), names=ind_names
        )

        start_date = dt.date.fromisoformat(rootgrp.variables[tcol].units.split()[-2])
        dfs = []
        for i in tqdm(range(out_var.shape[0])):
            dfs.append(
                pd.Series(
                    np.reshape(out_var[i : i + 1], geo_ind.shape[0]).data, index=geo_ind
                )
                .loc[lambda s: s != out_var._FillValue]
                .rename(out_desc)
                .reset_index()
                .assign(day=start_date + dt.timedelta(days=i))
            )
            if (i + 1) % 100 == 0:
                dump_out(dfs, i, out_desc, dump_dir)
                dfs = []
        if dfs:
            dump_out(dfs, i, out_desc, dump_dir)

In [ ]:
complete_df = pd.concat(
    (
        pd.concat(map(pd.read_parquet, gpl)).set_index(["NUTS_ID", "day"]).sort_index()
        for _, gpl in groupby(
            sorted(dump_dir.iterdir()), lambda p: p.name.split("-")[0]
        )
    ),
    axis=1,
)

In [ ]:
complete_df.to_parquet(dump_tmp_dir / "complete_eu.parquet")

In [ ]:
cyears = pd.to_datetime(complete_df.index.get_level_values("day")).year

In [ ]:
out_dir = dz.get_raw_data_path("yearly-nuts-weather-dumps")
out_dir.mkdir(exist_ok=True)

In [ ]:
for gid, gdf in tqdm(complete_df.groupby(cyears)):
    gdf.round(1).to_csv(out_dir / f"{gid}.csv.gz")

In [ ]:
nuts3_gdf.merge(
    complete_df.loc[(slice(None), dt.date(2013, 1, 11)), :].reset_index()
).plot("maximum_temperature", figsize=(15, 7), legend=True)